In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import os
from transformers import pipeline

# ====== Initialization ======
st.set_page_config(page_title="AI Debate System", page_icon="🤖")
st.title("🤖 Multi-Agent Debate System (Mistral Edition)")

st.write("Enter a topic and the AI agents will debate it (PRO vs CON) while a MODERATOR summarizes the result.")

# HuggingFace token
HF_TOKEN = "ENTER_HF_TOKEN"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN

# Load model
@st.cache_resource
def load_model():
    return pipeline(
        "text-generation",
        model="mistralai/Mistral-7B-Instruct-v0.2",
        torch_dtype="auto",
        device_map="auto"
    )

generator = load_model()

# ====== Agents ======
def agent_pro(topic):
    prompt = f"You are PRO. Give 3 strong arguments supporting the topic:\nTopic: {topic}\n"
    return generator(prompt, max_new_tokens=200, temperature=0.7)[0]["generated_text"]

def agent_con(topic):
    prompt = f"You are CON. Give 3 strong arguments against the topic:\nTopic: {topic}\n"
    return generator(prompt, max_new_tokens=200, temperature=0.7)[0]["generated_text"]

def agent_moderator(topic, pro, con):
    prompt = f"You are MODERATOR.\nTopic: {topic}\n\nPRO said:\n{pro}\n\nCON said:\n{con}\n\nCompare arguments and decide who is more convincing. Provide a short summary."
    return generator(prompt, max_new_tokens=250, temperature=0.5)[0]["generated_text"]

# ====== UI ======
topic = st.text_input("Enter debate topic:", "Should AI replace teachers?")

if st.button("Start Debate"):
    with st.spinner("Agents debating... Please wait"):
        pro = agent_pro(topic)
        con = agent_con(topic)
        mod = agent_moderator(topic, pro, con)

    st.subheader("🟢 PRO – Supporting Arguments")
    st.write(pro)

    st.subheader("🔴 CON – Opposing Arguments")
    st.write(con)

    st.subheader("⚖️ MODERATOR – Conclusion")
    st.write(mod)
